# 4.1 트랜스퍼 러닝 (ResNet)

- 작성자: [김찬란](https://github.com/seriousran)

## 4.1.2 ResNet 살펴보기

In [0]:
# 코드 4-1 케라스에 구현되어 있는 ResNet

keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet.ResNet101(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet.ResNet152(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet_v2.ResNet50V2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet_v2.ResNet101V2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet_v2.ResNet152V2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

## 4.1.3 인공신경망 내부 들여다보기

In [0]:
# 코드 4-2 가중치 시각화를 위한 함수들

import keras
import keras.backend as K
from keras.applications.resnet_v2 import ResNet50V2

def normalize(x):
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())


def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.25

    x += 0.5
    x = np.clip(x, 0, 1)

    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def process_image(x, former):
    return (x / 255 - 0.5) * 4 * former.std() + former.mean()


def visualize_layer(model,
                    layer_name,
                    step=1.,
                    epochs=15,
                    upscaling_steps=9,
                    upscaling_factor=1.2,
                    output_dim=(412, 412),
                    filter_range=(0, None)):

    def _generate_filter_image(input_img,
                               layer_output,
                               filter_index):
        s_time = time.time()

        loss = K.mean(layer_output[:, :, :, filter_index])

        grads = K.gradients(loss, input_img)[0]
        grads = normalize(grads)
        iterate = K.function([input_img], [loss, grads])

        intermediate_dim = tuple(
            int(x / (upscaling_factor ** upscaling_steps)) for x in output_dim)
        input_img_data = np.random.random(
            (1, intermediate_dim[0], intermediate_dim[1], 3))
        input_img_data = (input_img_data - 0.5) * 20 + 128

        for up in reversed(range(upscaling_steps)):
            for _ in range(epochs):
                loss_value, grads_value = iterate([input_img_data])
                input_img_data += grads_value * step

                if loss_value <= K.epsilon():
                    return None

            intermediate_dim = tuple(
                int(x / (upscaling_factor ** up)) for x in output_dim)
            img = deprocess_image(input_img_data[0])
            img = np.array(Image.fromarray(img).resize(intermediate_dim,
                                                           Image.BICUBIC))
            input_img_data = [process_image(img, input_img_data[0])]

        img = deprocess_image(input_img_data[0])
        e_time = time.time()
        print('Costs of filter {:3}: {:5.0f} ( {:4.2f}s )'.format(filter_index,
                                                                  loss_value,
                                                                  e_time - s_time))
        return img, loss_value

    def _draw_filters(filters, n=None):
        if n is None:
            n = int(np.floor(np.sqrt(len(filters))))

        filters.sort(key=lambda x: x[1], reverse=True)
        filters = filters[:n * n]

        MARGIN = 0
        width = n * output_dim[0] + (n - 1) * MARGIN
        height = n * output_dim[1] + (n - 1) * MARGIN
        stitched_filters = np.zeros((width, height, 3), dtype='uint8')

        for i in range(n):
            for j in range(n):
                img, _ = filters[i * n + j]
                width_margin = (output_dim[0] + MARGIN) * i
                height_margin = (output_dim[1] + MARGIN) * j
                stitched_filters[
                    width_margin: width_margin + output_dim[0],
                    height_margin: height_margin + output_dim[1], :] = img

        save_img('/content/drive/My Drive/0_colab/conv/resnet_{0:}_{1:}x{1:}.png'.format(layer_name, n), stitched_filters)
    assert len(model.inputs) == 1
    input_img = model.inputs[0]

    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

    output_layer = layer_dict[layer_name]
    assert isinstance(output_layer, Conv2D)

    filter_lower = filter_range[0]
    filter_upper = (filter_range[1]
                    if filter_range[1] is not None
                    else len(output_layer.get_weights()[1]))

    assert(filter_lower >= 0
           and filter_upper <= len(output_layer.get_weights()[1])
           and filter_upper > filter_lower)
    print('Compute filters {:} to {:}'.format(filter_lower, filter_upper))

    processed_filters = []
    for f in range(filter_lower, filter_upper):
        img_loss = _generate_filter_image(input_img, output_layer.output, f)

        if img_loss is not None:
            processed_filters.append(img_loss)

        if len(processed_filters) == 9:
          break

    print('{} filter processed.'.format(len(processed_filters)))
    _draw_filters(processed_filters)

In [0]:
resnet_v2 = ResNet50V2(include_top=False, weights='imagenet',
                       input_shape = (32,32,3))
visualize_layer(resnet_v2, 'conv1_conv')

Compute filters 0 to 64
Costs of filter   0:    38 ( 6.71s )
Costs of filter   1:   168 ( 6.59s )
Costs of filter   5:    25 ( 6.57s )
Costs of filter   7:    18 ( 6.80s )
Costs of filter   8:    36 ( 6.78s )
Costs of filter  10:   160 ( 6.72s )
Costs of filter  13:     8 ( 6.70s )
Costs of filter  14:     7 ( 6.69s )
Costs of filter  15:    24 ( 6.69s )
Costs of filter  16:    45 ( 6.79s )
Costs of filter  17:    28 ( 6.82s )
Costs of filter  20:    29 ( 6.74s )
Costs of filter  24:    12 ( 6.82s )
Costs of filter  27:    15 ( 6.68s )
Costs of filter  30:     5 ( 6.90s )
Costs of filter  31:   110 ( 6.75s )
Costs of filter  32:    16 ( 6.67s )
Costs of filter  37:    51 ( 6.72s )
Costs of filter  40:    28 ( 6.90s )
Costs of filter  41:     7 ( 6.77s )
Costs of filter  44:   155 ( 7.01s )
Costs of filter  45:    16 ( 6.75s )
Costs of filter  46:    16 ( 6.85s )
Costs of filter  49:   153 ( 6.90s )
Costs of filter  58:    59 ( 6.90s )
25 filter processed.


In [0]:
visualize_layer(resnet_v2, 'conv2_block2_3_conv')

Compute filters 0 to 256
Costs of filter   3:   133 ( 7.88s )
Costs of filter   7:    28 ( 8.02s )
Costs of filter   8:    26 ( 7.93s )
Costs of filter  14:    65 ( 8.04s )
Costs of filter  22:   102 ( 8.16s )
Costs of filter  25:   251 ( 8.10s )
Costs of filter  31:    56 ( 8.45s )
Costs of filter  34:   103 ( 8.50s )
Costs of filter  35:    95 ( 8.58s )
Costs of filter  36:    50 ( 8.43s )
Costs of filter  40:    41 ( 8.48s )
Costs of filter  43:    15 ( 8.71s )
Costs of filter  46:   113 ( 8.86s )
Costs of filter  47:   176 ( 8.66s )
Costs of filter  51:    91 ( 8.71s )
Costs of filter  53:    87 ( 9.03s )
Costs of filter  63:    54 ( 9.20s )
Costs of filter  66:    50 ( 9.20s )
Costs of filter  74:    44 ( 9.45s )
Costs of filter  77:    94 ( 9.54s )
Costs of filter  79:   122 ( 9.31s )
Costs of filter  82:    91 ( 9.27s )
Costs of filter  84:    43 ( 9.60s )
Costs of filter  85:    32 ( 9.46s )
Costs of filter 105:   132 ( 9.77s )
25 filter processed.


In [0]:
visualize_layer(resnet_v2, 'conv2_block3_3_conv')

Compute filters 0 to 256
Costs of filter   7:    47 ( 10.54s )
Costs of filter   9:    38 ( 10.63s )
Costs of filter  10:    47 ( 10.71s )
Costs of filter  12:   132 ( 10.72s )
Costs of filter  14:    81 ( 10.61s )
Costs of filter  16:    50 ( 10.90s )
Costs of filter  17:   151 ( 11.24s )
Costs of filter  24:    55 ( 11.15s )
Costs of filter  31:   112 ( 11.17s )
Costs of filter  35:    73 ( 11.38s )
Costs of filter  40:    50 ( 11.42s )
Costs of filter  41:    34 ( 11.65s )
Costs of filter  45:   150 ( 11.50s )
Costs of filter  47:   181 ( 11.73s )
Costs of filter  48:    58 ( 11.69s )
Costs of filter  49:    55 ( 11.94s )
Costs of filter  51:   162 ( 12.02s )
Costs of filter  68:    82 ( 12.33s )
Costs of filter  72:    83 ( 12.44s )
Costs of filter  73:   129 ( 12.58s )
Costs of filter  74:    68 ( 12.39s )
Costs of filter  79:    20 ( 12.51s )
Costs of filter  81:   106 ( 12.53s )
Costs of filter  84:    12 ( 12.73s )
Costs of filter  85:    52 ( 12.95s )
25 filter processed.


In [0]:
visualize_layer(resnet_v2, 'conv3_block1_3_conv')

Compute filters 0 to 512
Costs of filter   4:    55 ( 13.57s )
Costs of filter   7:    26 ( 13.35s )
Costs of filter  14:    15 ( 13.62s )
Costs of filter  16:    66 ( 13.75s )
Costs of filter  18:    49 ( 13.95s )
Costs of filter  19:    31 ( 13.83s )
Costs of filter  31:    25 ( 14.64s )
Costs of filter  32:    48 ( 14.38s )
Costs of filter  39:    83 ( 14.56s )
Costs of filter  44:    40 ( 15.07s )
Costs of filter  47:    32 ( 14.98s )
Costs of filter  50:    62 ( 15.16s )
Costs of filter  55:    20 ( 15.31s )
Costs of filter  63:    21 ( 15.77s )
Costs of filter  66:    31 ( 15.43s )
Costs of filter  70:    81 ( 15.64s )
Costs of filter  83:    52 ( 16.47s )
Costs of filter  84:   107 ( 16.30s )
Costs of filter  85:    80 ( 16.56s )
Costs of filter  87:    16 ( 16.51s )
Costs of filter  89:    45 ( 16.65s )
Costs of filter  91:    55 ( 16.72s )
Costs of filter  92:   107 ( 16.49s )
Costs of filter  95:    58 ( 16.75s )
Costs of filter  98:    39 ( 16.97s )
25 filter processed.


In [0]:
visualize_layer(resnet_v2, 'conv3_block4_3_conv')

Compute filters 0 to 512
Costs of filter   1:   169 ( 18.37s )
Costs of filter   3:    43 ( 18.60s )
Costs of filter   7:    35 ( 19.17s )
Costs of filter  12:    45 ( 18.92s )
Costs of filter  29:    45 ( 19.96s )
Costs of filter  32:    24 ( 20.40s )
Costs of filter  34:    46 ( 20.53s )
Costs of filter  38:    77 ( 20.58s )
Costs of filter  39:    64 ( 20.96s )
Costs of filter  41:   150 ( 20.97s )
Costs of filter  47:   187 ( 21.23s )
Costs of filter  50:    16 ( 21.48s )
Costs of filter  51:    45 ( 21.68s )
Costs of filter  58:    29 ( 22.04s )
Costs of filter  66:    76 ( 22.46s )
Costs of filter  68:    57 ( 22.61s )
Costs of filter  73:    69 ( 22.79s )
Costs of filter  75:    41 ( 23.23s )
Costs of filter  76:    34 ( 23.09s )


In [0]:
visualize_layer(resnet_v2, 'conv4_block1_3_conv')

Compute filters 0 to 1024
Costs of filter   1:    47 ( 5.51s )
Costs of filter   7:    13 ( 4.56s )
Costs of filter  10:    47 ( 4.75s )
Costs of filter  12:    17 ( 4.90s )
Costs of filter  16:    39 ( 5.18s )
Costs of filter  18:    39 ( 5.35s )
Costs of filter  19:    14 ( 5.43s )
Costs of filter  25:    43 ( 5.76s )
Costs of filter  26:    70 ( 5.81s )
Costs of filter  27:    21 ( 6.17s )
Costs of filter  29:    88 ( 6.01s )
Costs of filter  33:    19 ( 6.48s )
Costs of filter  36:    51 ( 6.68s )
Costs of filter  40:    15 ( 6.93s )
Costs of filter  41:    10 ( 6.96s )
Costs of filter  43:    35 ( 7.17s )
Costs of filter  57:    25 ( 8.09s )
Costs of filter  64:    23 ( 8.92s )
Costs of filter  65:    30 ( 8.60s )
Costs of filter  71:    11 ( 9.04s )
Costs of filter  76:    30 ( 9.44s )
Costs of filter  83:     7 ( 9.88s )
Costs of filter  89:    43 ( 10.29s )
Costs of filter  93:    78 ( 10.55s )
Costs of filter 104:    56 ( 11.38s )
25 filter processed.


In [0]:
visualize_layer(resnet_v2, 'conv4_block6_3_conv')

Compute filters 0 to 1024
Costs of filter   3:    35 ( 58.30s )
Costs of filter  47:    61 ( 61.91s )
Costs of filter  56:   378 ( 64.66s )
Costs of filter 274:   103 ( 90.64s )
Costs of filter 319:    21 ( 95.52s )


In [0]:
visualize_layer(resnet_v2, 'conv5_block1_3_conv')

In [0]:
visualize_layer(resnet_v2, 'conv5_block3_3_conv')